# PREPARACIÓN PARA EL WEB SCRAPING

### Instalación e importación de las librerías necesarias

In [ ]:
# Se instalan y se importan las librerías necesarias
!pip install gspread
!pip install pymongo
!pip install instagrapi
!pip install facebook-scraper
import gspread
import re
import random
import time
from facebook_scraper import get_posts
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from instagrapi import Client

### Extracción de los datos desde google sheets

In [ ]:
# Abrir documento de google sheets
gc = gspread.service_account(filename='') # Archivo JSON descargado desde Google Sheets que permite realizar la conexión
sh = gc.open_by_url('') # Url hacia el archivo de Google Sheets
worksheet = sh.get_worksheet(0)
records = worksheet.get_all_records() # Se guardan los datos de los clientes en la variable, en una forma de lista de diccionarios

### Inicio de sesión en Facebook

In [ ]:
# Se ingresa el correo y la contraseña de la cuenta de facebook que usaremos para scrapear
user_fb = "" # Email
pass_fb = "" # Contraseña
credentials = (user_fb, pass_fb)

### Inicio de sesión en Instagram

In [ ]:
# Iniciar sesión en Instagram
user_insta = "" # Username
pass_insta = "" # Contraseña
cl = Client()
cl.login(user_insta, pass_insta)

# WEB SCRAPING



### Loop

In [ ]:
# Se generan patrones para extraer el username de los link de facebook e instagram
patron_fb = r"(?:facebook\.com\/|@)([a-zA-Z0-9._]+)"
patron_insta = r"(?:instagram\.com\/|@)([a-zA-Z0-9._]+)"

# Se inicializa un ciclo for para cada cliente que hay en el google sheet
for cliente in records:

    # Se extrae el username del link de facebook del cliente
    coincidencias_fb = re.findall(patron_fb, cliente['Link del Facebook de tu emprendimiento'])
    if coincidencias_fb:
        fb_username = coincidencias_fb[0] # ID de facebook del emprendimiento

    # Se extrae el username del link de instagram del cliente
    coincidencias_insta = re.findall(patron_insta, cliente['Link del Instagram de tu emprendimiento'])
    if coincidencias_insta:
        insta_username = coincidencias_insta[0] # ID de instagram del emprendimiento

    # Se genera una excepción para scrapear los posts de facebook, si falla, lo vuelve a intentar,
    # ya que se cierra la sesión después de un tiempo y avienta error al intentar iniciarla de nuevo, esto se resuelve intentandolo una segunda vez
    try:
        posts = get_posts(account=fb_username, pages=3, credentials=credentials) # Se ingresa el username, la cantidad de páginas (scrolleadas) que queremos scrapear
                                                                                 # y las credenciales con las que iniciaremos sesión en la función get_posts
    except:
        posts = get_posts(account=fb_username, pages=3, credentials=credentials)
        posts = [post for post in posts]
    else:
        posts = [post for post in posts]

    # Si la función get_posts genera una lista vacia, significa que la página de facebook no existe,
    # por lo que el resultado de las métricas pasa a ser "N/A"
    if posts == []:
        cliente['Fecha de la última publicación en Facebook'] = 'N/A'
        cliente['Promedio de días entre publicaciones en Facebook'] = 'N/A'
        cliente['Promedio de likes por publicación en Facebook'] = 'N/A'
        cliente['Promedio de comentarios por publicación en Facebook'] = 'N/A'
        cliente['Promedio de shares por publicación en Facebook'] = 'N/A'

    # Si la página de facebook existe, se genera un ciclo for para extraer las métricas
    else:

        # Se generan listas vacías para guardar la cantidad de likes, comentarios, shares, fechas por post y una lista de diferencias entre las fechas de los posts
        likes_fb = []
        comments_fb = []
        shares_fb = []
        fechas_fb = []
        diferencias_fb = []

        # Se genera un ciclo for que ingresa los datos por post a las listas vacías anteriormente creadas
        for i in posts:
            likes_fb.append(i['likes'])
            comments_fb.append(i['comments'])
            shares_fb.append(i['shares'])
            fechas_fb.append(i['time'])

        fechas_fb = sorted(fechas_fb) # Se ordenan las fechas

        # Se genera un ciclo for que ingresa la diferencia de días entre cada post a la lista vacía
        for i in range(1, len(fechas_fb)):
            diferencia_fb = (fechas_fb[i] - fechas_fb[i - 1]).days
            diferencias_fb.append(diferencia_fb)

        # Se generan las métricas a partir de los datos scrapeados
        prom_frecuencia_fb = round(sum(diferencias_fb) / len(diferencias_fb), 2) # Promedio de las frecuencias entre los posts (mientras mas bajo, mejor)
        prom_likes_fb = round(sum(likes_fb)/len(likes_fb), 2)                    # Promedio de cantidad de likes
        prom_comments_fb = round(sum(comments_fb)/len(comments_fb), 2)           # Promedio de cantidad de comentarios
        prom_shares_fb = round(sum(shares_fb)/len(shares_fb), 2)                 # Promedio de cantidad de shares
        fecha_ult_publi_fb = posts[0]['time'].strftime("%d-%m-%Y")               # Fecha de la publicación mas reciente

        # Se añaden las métricas al diccionario del cliente
        cliente['Fecha de la última publicación en Facebook'] = fecha_ult_publi_fb
        cliente['Promedio de días entre publicaciones en Facebook'] = prom_frecuencia_fb
        cliente['Promedio de likes por publicación en Facebook'] = prom_likes_fb
        cliente['Promedio de comentarios por publicación en Facebook'] = prom_comments_fb
        cliente['Promedio de shares por publicación en Facebook'] = prom_shares_fb

    # Se genera una excepción para scrapear la información de instagram, si falla significa que el usuario de instagram no existe o que nos han baneado nuestra cuenta,
    # por lo que el resultado de las métricas pasa a ser "N/A"
    try:
        cliente_insta_info = cl.user_info_by_username(insta_username).dict()
    except:
        cliente['Cantidad de seguidores en Instagram'] = 'N/A'
        cliente['Cantidad de publicaciones en Instagram'] = 'N/A'
        cliente['Fecha de la última publicación en Instagram'] = 'N/A'
        cliente['Promedio de días entre publicaciones en Instagram'] = 'N/A'
        cliente['Promedio de likes por publicación en Instagram'] = 'N/A'
        cliente['Promedio de comentarios por publicación en Instagram'] = 'N/A'

    # Si el usuario existe, se generan las métricas
    else:
        cant_posts_insta = cliente_insta_info['media_count']         # Cantidad de posts
        cant_followers_insta = cliente_insta_info['follower_count']  # Cantidad de seguidores

        cliente_insta_id = cl.user_info_by_username(insta_username).dict()['pk'] # ID de la página de instagram del cliente
        cliente_insta_posts = cl.user_medias_v1(cliente_insta_id, amount=5)      # Se ingresa el ID de la página de instagram a scrapear y la cantidad de posts a scrapear

        # Se generan listas vacías para guardar la cantidad de likes, comentarios, fechas por post y una lista de diferencias entre las fechas de los posts
        likes_insta = []
        comments_insta = []
        fechas_insta = []
        diferencias_insta = []

        # Se genera un ciclo for que ingresa los datos por post a las listas vacías anteriormente creadas
        for insta_post in cliente_insta_posts:
            likes_insta.append(insta_post.dict()['like_count'])
            comments_insta.append(insta_post.dict()['comment_count'])
            fechas_insta.append(insta_post.dict()['taken_at'])

        fechas_insta = sorted(fechas_insta) # Se ordenan las fechas

        # Se genera un ciclo for que ingresa la diferencia de días entre cada post a la lista vacía
        for i in range(1, len(fechas_insta)):
            diferencia_insta = (fechas_insta[i] - fechas_insta[i - 1]).days
            diferencias_insta.append(diferencia_insta)

        # Se generan las métricas a partir de los datos scrapeados
        prom_likes_insta = round(sum(likes_insta)/len(likes_insta), 2)                          # Promedio de cantidad de likes
        prom_comments_insta = round(sum(comments_insta)/len(comments_insta), 2)                 # Promedio de cantidad de comentarios
        prom_frecuencia_insta = round(sum(diferencias_insta) / len(diferencias_insta), 2)       # Promedio de las frecuencias entre los posts (mientras mas bajo, mejor)
        fecha_ult_publi_insta = cliente_insta_posts[0].dict()['taken_at'].strftime("%d-%m-%Y")  # Fecha de la publicación mas reciente

        # Se añaden las métricas al diccionario del cliente
        cliente['Cantidad de seguidores en Instagram'] = cant_followers_insta
        cliente['Cantidad de publicaciones en Instagram'] = cant_posts_insta
        cliente['Fecha de la última publicación en Instagram'] = fecha_ult_publi_insta
        cliente['Promedio de días entre publicaciones en Instagram'] = prom_frecuencia_insta
        cliente['Promedio de likes por publicación en Instagram'] = prom_likes_insta
        cliente['Promedio de comentarios por publicación en Instagram'] = prom_comments_insta

    print(cliente)

    # Para evitar que baneen las cuentas de facebook e instagram, se pausa el ciclo for durante 30 mins, añadiendole de 1 a 15 mins extras aleatoriamente
    tiempo_pausa_extra= random.uniform(60, 900)
    time.sleep(1800 + tiempo_pausa_extra)

### Recomendaciones

* Si el resultado de las métricas de Instagram del emprendimiento de algún cliente es "N/A" se trata de que esa página de Instagram no existe, si esto persiste con los siguientes clientes en la lista, se debe revisar la cuenta que se esté utilizando para scrapear, ya que probablemente ha sido baneada. La mayor cantidad de páginas que se lograron scrapear sin ser baneados fueron 27.
* Se pueden probar distintos tiempos de pausa para ver cual es el mas conveniente para la cantidad de registros que se tienen, así también, para aumentar la cantidad de páginas que se logren scrapear sin ser baneados.
* Se deberán crear cuentas de Instagram constantemente, probablemente una cada 2 días o las que sean necesarias. Igualmente si llegan a banear la cuenta de Facebook utilizada, se deberán crear más cuentas de Facebook.
* Utilizar una VPN puede llegar a ser contradictorio ya que Facebook e Instagram detectan cuando el usuario está utilizando una VPN y lo marcan como actividad sospechosa. Al igual que no banean la IP del usuario, si no las cuentas, por lo tanto no sirve de mucho una VPN.

# MONGODB

### Conexión con MongoDB

In [ ]:
# Se debe iniciar sesion en MongoDB y añadir la dirección IP si es que lo pide.

# Se genera la variable uri, en donde hay un usuario, una contraseña, y un cluster en donde ingresaremos los datos
uri = "" # Esta variable se genera en MongoDB el la sección de "Connect"

# Crear un nuevo cliente y conectar con el servidor
client = MongoClient(uri, server_api=ServerApi('1'))

# Hacer un ping para confirmar la conexión
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

### Selección de base de datos

In [ ]:
# Seleccionar base de datos
db = client[''] # Colocar base de datos
collection = db[''] # Colocar colección

### Inserción de datos

In [ ]:
# Se insertan los clientes a la base de datos en MongoDB
for cliente in records:
    collection.insert_one(cliente)